# LangChain Chain Types: A Comprehensive Comparison

LangChain provides several `chain_type` strategies to process, summarize, and interact with multiple documents to achieve various tasks like summarization, question answering, and more. Below is a detailed comparison of the common chain types used in LangChain.

## 1. Map-Reduce
The `map-reduce` approach is a well-known distributed data processing strategy that has been adapted for language model tasks involving multiple documents or large data.

- **How It Works**:
  - **Map Phase**: Each document is independently processed to generate an intermediate result. For example, the model answers a question or creates a summary for each document separately.
  - **Reduce Phase**: The results from the map phase are then aggregated or summarized to create a final answer.

- **Pros**:
  - **Scalability**: It can process large volumes of documents since each document is handled independently. This makes it well-suited for large-scale datasets.
  - **Parallel Processing**: The mapping phase can be parallelized, making it efficient for handling many documents.

- **Cons**:
  - **Context Loss**: Since each document is processed in isolation, there may be a lack of overall context or coherence when aggregating results.
  - **Reduce Complexity**: The reduce step must be well-defined to merge diverse responses effectively, which can be challenging for complex questions.

- **Best Use Cases**:
  - Summarizing a large number of documents.
  - Question answering from a broad set of documents.

## 2. Refine
The `refine` chain type is an iterative method that refines the output by taking the previous answer and adding to it with the context from each subsequent document.

- **How It Works**:
  - The first document is processed to generate an initial answer.
  - Each subsequent document is used to refine the previous answer, incorporating additional context and modifying the response iteratively.

- **Pros**:
  - **Comprehensive Context**: Since each document is used to iteratively improve the answer, it ensures that every document contributes to the final output.
  - **Logical Continuity**: The approach allows the response to grow and evolve logically, building on the context from previous documents.

- **Cons**:
  - **Sequential Processing**: It cannot be easily parallelized, as each step depends on the previous one.
  - **Time-Consuming**: The iterative refinement process can be slower, especially when dealing with a large number of documents.

- **Best Use Cases**:
  - Long-form question answering where each document contributes additional details to the overall answer.
  - Cases where maintaining a logical, evolving context is crucial.

## 3. Map-Rerank
The `map-rerank` approach is similar to `map-reduce` but with a focus on ranking rather than merging the responses.

- **How It Works**:
  - **Map Phase**: Each document is processed independently to generate a response.
  - **Rerank Phase**: Each response is then scored based on its relevance or quality, and the highest-ranked response is selected.

- **Pros**:
  - **Relevance Focus**: Produces the most relevant response by selecting the highest-scoring result.
  - **Efficiency**: Avoids aggregating diverse answers, simplifying the final output.

- **Cons**:
  - **Loss of Potentially Useful Context**: Some information from lower-ranked responses might be ignored, leading to loss of context.
  - **Ranking Quality Dependence**: The final outcome depends on how effectively the responses are scored and ranked.

- **Best Use Cases**:
  - Retrieval-based tasks where a single, best answer is needed.
  - Scenarios where focusing on the most relevant information is more important than combining all responses.

## 4. Stuff
The `stuff` approach is the simplest chain type that directly concatenates all the documents and feeds them to the model in a single pass.

- **How It Works**:
  - All the documents are concatenated to form one large input that is then fed to the model for a single summarization or answer generation.

- **Pros**:
  - **Full Context**: Since all the documents are concatenated, the model gets access to the entire context at once.
  - **Simplicity**: Very straightforward and easy to implement.

- **Cons**:
  - **Token Limit Constraints**: The model has a token limit, which means it can only handle a small number of documents at a time if they are large.
  - **Scalability Issues**: It is not suitable for large-scale document processing because of token size limitations.

- **Best Use Cases**:
  - Summarizing a few short documents that can easily fit into the model’s token limit.
  - Cases where the complete context from all documents is important for the final answer.

## Summary of Chain Types

| Chain Type      | How It Works                      | Pros                                    | Cons                                     | Best Use Cases                         |
|-----------------|-----------------------------------|-----------------------------------------|------------------------------------------|----------------------------------------|
| **Map-Reduce**  | Independent summaries then merged | **Scalable**, **Parallelizable**        | **Context Loss**, **Complex Reduction**  | Summarizing large datasets, Q&A        |
| **Refine**      | Iterative refinement of the answer | **Comprehensive**, **Context Continuity** | **Sequential**, **Slow**                 | Long-form Q&A, Building iterative context |
| **Map-Rerank**  | Independent responses, then ranked | **Focus on Relevance**, **Efficient**   | **Potential Context Loss**               | Best-answer retrieval                  |
| **Stuff**       | Concatenates all docs, single pass | **Full Context**, **Simple**            | **Token Limit**, **Not Scalable**        | Summarizing few short docs, Full context required |

## Choosing the Right Chain Type
- **If you have many documents and need an overview**: Use **Map-Reduce**. It is efficient for summarizing multiple documents independently and then combining them.
- **If you need to iteratively build on context from each document**: Choose **Refine**. It helps in scenarios where logical evolution of the answer is required, providing a comprehensive view by iteratively refining the output.
- **If relevance is the primary factor**: **Map-Rerank** is ideal when you need the single most relevant response from multiple documents. It helps in picking the top-quality content efficiently.
- **If the number of documents is small, and all content needs to be considered together**: Use **Stuff**. It is useful when you can fit all documents into a single prompt, ensuring the full context is considered.

## Combining Chain Types
- **For Maximum Comprehensiveness**: Combine `Map-Reduce` with `Refine` for cases where you need both a broad overview and an iterative improvement to achieve depth.
- **For Relevance and Context**: Use `Map-Rerank` to select the most relevant content, then use `Refine` to enhance and build upon it for a coherent output.

Each chain type has strengths that align with different document processing needs, so understanding these differences can help in selecting the best chain for a specific use case or combining them effectively for maximum benefit.


In [2]:
%pip install langchain tiktoken

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.2")

In [4]:
%pip install --upgrade --quiet  youtube-transcript-api
%pip install --upgrade --quiet  pytube

from langchain_community.document_loaders import YoutubeLoader

loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=BqbIRiYXd3U", add_video_info=False
)
# load the youtube video caption into Documents
docs = loader.load()
# check how many characters in the doc and some content
len(docs[0].page_content), docs[0].page_content[:300], len(docs)
text = docs[0].page_content
summary = llm.invoke(f"Give me a summary of the text below: {text}.")
print(summary)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# create splits from content
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)
# check length
len(split_docs), len(docs), len(split_docs[0].page_content), len(docs[0].page_content)


(19, 1, 4655, 88209)

In [6]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm, chain_type="refine")
refineResult = chain.run(split_docs)
summaries = []
summaries.append(refineResult)

/var/folders/bv/01rc_d8n7tdgkwp7g9bvp4p40000gn/T/ipykernel_3596/1726956926.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  refineResult = chain.run(split_docs)


In [7]:
chain = load_summarize_chain(llm, chain_type="stuff")
stuffResult = chain.run(split_docs)
summaries.append(stuffResult)

In [8]:
## shorter summary
chain = load_summarize_chain(llm, chain_type="map_reduce") # another option is map-rerank
map_reduceResult = chain.run(split_docs)
summaries.append(map_reduceResult)

Token indices sequence length is longer than the specified maximum sequence length for this model (3039 > 1024). Running this sequence through the model will result in indexing errors


In [9]:
## shorter summary
chain = load_summarize_chain(llm, chain_type="map_rerank") # another option is map-rerank
map_reduceResult = chain.run(split_docs)
summaries.append(map_reduceResult)

ValueError: Got unsupported chain type: map_rerank. Should be one of dict_keys(['stuff', 'map_reduce', 'refine'])

In [ ]:
# tl;dr generator
from langchain_community.document_loaders import TextLoader

# for summary in summaries:
#     with open(f'temp_texts{summary[0]}.txt', 'w') as f:


with open('temp_texts.txt', 'w') as f:
    for summary in summaries:
        f.write(summary + '\n')

text_loader = TextLoader('temp_texts.txt')
summary_docs = text_loader.load();
print(summary_docs)
split_summaries = text_splitter.split_documents(summary_docs)

final_chain = load_summarize_chain(llm, chain_type="stuff")
tldr = final_chain.run(split_summaries)

In [ ]:
# print(refineResult)
print(summaries)
# print(docs)
# len(tldr), len(refineResult), len(map_reduceResult)
# print(tldr, refineResult, map_reduceResult)